In [1]:
import sys
import os
import time

sys.path.insert(0, '..')
sys.path.insert(0, '../../futile/python')

package = 'URL'

import BigDFT
BigDFT.Logger.path = os.path.join(os.getcwd(), f'BigDFT_test{package}.yaml')
BigDFT.Logger.level = 'debug'
BigDFT.Logger.overwrite = True
BigDFT.Logger.info(f'{package} unit testing begins')

import BigDFT.RemoteRunners as r

from BigDFT.URL import URL

remote_dir = os.path.join(os.getcwd(), f'fake_remote_{package}')
local_dir = os.path.join(os.getcwd(), f'fake_local_{package}')

print(f'running with:\nremote dir: {remote_dir}\nlocal dir: {local_dir}')

running with:
remote dir: /home/test/bigdft-suite/PyBigDFT/Tests/fake_remote_URL
local dir: /home/test/bigdft-suite/PyBigDFT/Tests/fake_local_URL


In [2]:
from BigDFT.URL import URL

### Test BigDFT.URL

In [3]:
# NBVAL_CHECK_OUTPUT
# basic instantisation
u = URL(user = 'test', host = '127.0.0.1', port = 1800)

print(u.user)
print(u.host)
print(u.port)
print(u.local)
print(u.is_tunnel)

test
127.0.0.1
1800
False
True


In [4]:
# NBVAL_CHECK_OUTPUT
# from_string method
u = URL.from_string('test@127.0.0.1', rsync='auq')

print(u.user)
print(u.host)
print(u.local)
print(u.is_tunnel)
print(u.rsync_flags)

test
127.0.0.1
False
False
-auq


In [5]:
# NBVAL_CHECK_OUTPUT
# update values
u.host = 'new'
u.port = 1800
print(u.local)
print(u.is_tunnel)

False
True


In [6]:
# NBVAL_CHECK_OUTPUT
# 'local' test
u = URL()

print(u.local)

True


In [7]:
# test that test_connection properly resets permissions
u._permissions['rsync'] = 'cp fallback'
u._permissions['garbage'] = 'delete me'
ctest = u.test_connection(verbose=False)

In [8]:
# NBVAL_CHECK_OUTPUT
print(ctest)

True


In [9]:
# NBVAL_CHECK_OUTPUT
permissions = u.permissions
# need to pop options which would have differences in the CI
permissions.pop('latency')
permissions.pop('local_root')
print(permissions)

{'rsync': 'cp fallback', 'garbage': 'delete me'}


In [10]:
# NBVAL_CHECK_OUTPUT
# rsync tests. This cell is designed to also work (manually) enabling a remote connection
# setup
BigDFT.Logger.info('testing rsync behaviour')
u.test_connection(verbose=False)
u.ensure_dir(f'{local_dir}', force_local=True)
u.ensure_dir(f'{remote_dir}')
u.cmd(f'rm -r {remote_dir}/*', suppress_warn=True)
u.cmd(f'rm -r {local_dir}/*', suppress_warn=True)

u.cmd(f'touch {local_dir}/test_{{1,2,3}}', local=True)

BigDFT.Logger.info('PUSH: MULTIPLE')
# TESTING BEGINS: PUSH MULTIPLE
# multiple files
u.cmd(f'rm -r {remote_dir}/test*', suppress_warn=True)
u.rsync(['test_1','test_2','test_3'],
       origin=f'{local_dir}',
       target=f'{remote_dir}',
       push=True)

after = u.ls(f'{remote_dir}')
print('push multiple:', after)

# TESTING BEGINS: PULL MULTIPLE
BigDFT.Logger.info('PULL: MULTIPLE')
u.cmd(f'rm -r {local_dir}/test*', suppress_warn=True)
u.rsync(['test_1','test_2','test_3'],
       origin=f'{local_dir}',
       target=f'{remote_dir}',
       push=False)

after = u.ls(f'{local_dir}', local=True)
print('pull multiple:', after)

# TESTING BEGINS: PUSH SINGLE
BigDFT.Logger.info('PUSH: SINGLE')
u.cmd(f'rm -r {remote_dir}/test*', suppress_warn=True)
u.rsync(['test_1'],
       origin=f'{local_dir}',
       target=f'{remote_dir}',
       push=True)

after = u.ls(f'{remote_dir}')
print('push single:', after)

# TESTING BEGINS: PULL SINGLE
BigDFT.Logger.info('PULL: SINGLE')
u.cmd(f'rm -r {local_dir}/test*', suppress_warn=True, local=True)
u.rsync(['test_1'],
       origin=f'{local_dir}',
       target=f'{remote_dir}',
       push=False)

after = u.ls(f'{local_dir}', local=True)
print('pull single:', after)

# again, with cp_fallback forced
BigDFT.Logger.info('force enabling cp_fallback and repeating tests')
if u.local:
    u._permissions['rsync'] = u._rsync_cp_fallback
    u.cmd(f'touch {local_dir}/test_{{1,2,3}}')

    # TESTING BEGINS: PUSH MULTIPLE (CP)
    BigDFT.Logger.info('CP PUSH: MULTIPLE')
    u.cmd(f'rm -r {remote_dir}/test*', suppress_warn=True)
    u.rsync(['test_1','test_2','test_3'],
           origin=f'{local_dir}',
           target=f'{remote_dir}',
           push=True)

    after = u.ls(f'{remote_dir}')
    print('push multiple (cp)', after)

    # TESTING BEGINS: PULL MULTIPLE (CP)
    BigDFT.Logger.info('CP PULL: MULTIPLE')
    u.cmd(f'rm -r {local_dir}/test*', suppress_warn=True)
    u.rsync(['test_1','test_2','test_3'],
           origin=f'{local_dir}',
           target=f'{remote_dir}',
           push=False)

    after = u.ls(f'{local_dir}')
    print('pull multiple (cp)', after)

    # TESTING BEGINS: PUSH SINGLE (CP)
    BigDFT.Logger.info('CP PUSH: SINGLE')
    u.cmd(f'rm -r {remote_dir}/test*', suppress_warn=True)

    u.rsync(['test_1'],
           origin=f'{local_dir}',
           target=f'{remote_dir}',
           push=True)

    after = u.ls(f'{remote_dir}')
    print('push single (cp)', after)

    # TESTING BEGINS: PULL SINGLE (CP)
    BigDFT.Logger.info('CP PULL: SINGLE')
    u.cmd(f'rm -r {local_dir}/test*', suppress_warn=True)

    u.rsync(['test_1'],
           origin=f'{local_dir}',
           target=f'{remote_dir}',
           push=False)

    after = u.ls(f'{local_dir}')
    print('pull single (cp)', after)

push multiple: ['test_1', 'test_2', 'test_3']
pull multiple: ['test_1', 'test_2', 'test_3']
push single: ['test_1']
pull single: ['test_1']
push multiple (cp) ['test_1', 'test_2', 'test_3']
pull multiple (cp) ['test_1', 'test_2', 'test_3']
push single (cp) ['test_1']
pull single (cp) ['test_1']


In [11]:
# NBVAL_CHECK_OUTPUT
# Utils testing
BigDFT.Logger.info('utils testing begins')
u.cmd('mkdir ./scancheck')

files = ['test_1',
        'test_2',
        'test_3',
        'test_4']

for file in files[:-1]:
    u.utils.touch(f'./scancheck/{file}')
    
presence_list = u.utils.search_folder(files, './scancheck')
presence_one = u.utils.search_folder(files[0], './scancheck')

u.cmd('rm -r ./scancheck')

print(presence_list)
print(presence_one)

{'test_1': True, 'test_2': True, 'test_3': True, 'test_4': False}
{'test_1': True}
